In [15]:
from cleantext import clean
from pandarallel import pandarallel

In [16]:
import pandas as pd
import numpy as np
import databricks.koalas as ks
from pyspark.sql import SparkSession
import os
import pickle

In [17]:
from tqdm.notebook import tqdm 

In [18]:
spark_session = SparkSession \
    .builder \
    .appName("app") \
    .master("local[50]") \
    .config("spark.driver.memory","40G")\
    .getOrCreate();

# path

In [19]:
data_path = ''

In [20]:
filtered_dir = ''

In [21]:
# path = os.path.join(filtered_dir,'BoDeGHa_data_onefile/BoDeGHa_data.csv')
# comments = pd.read_csv(path, quotechar='"', escapechar='\\')
# # comments = comments[comments['repo_id']==str(repository)].drop(columns=['repo_id'])

In [22]:
# comments.head()

In [23]:
# comments[comments['repo_id']==213].drop(columns=['repo_id'])

# format text file for easier read (run this on .py)

In [25]:
# pandarallel.initialize(nb_workers=50)

INFO: Pandarallel will run on 50 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
# header_list = ["pull_request_id","user_id","comment_id","position","body","commit_id","created_at"]
# pull_request_comments = pd.read_csv(os.path.join(data_path,'pull_request_comments.csv'), header=None, names=header_list, quotechar='"', escapechar='\\')
# pull_request_comments['body'] = pull_request_comments.parallel_apply(lambda row: clean(row['body'], no_line_breaks=True),axis = 1)
# pull_request_comments['created_at'] =  pd.to_datetime(pull_request_comments['created_at'], format='%Y-%m-%d %H:%M:%S')
# pull_request_comments.to_csv(os.path.join(filtered_dir,'easy_read/pull_request_comments2.csv'), quotechar='"', escapechar='\\')
# pull_request_comments = ks.from_pandas(pull_request_comments)
# pull_request_comments.to_csv(os.path.join(filtered_dir,'easy_read/pull_request_comments'))

In [9]:
# header_list = ["id" ,"url" ,"owner_id" ,"name" ,"description" ,"language" ,"created_at" ,"forked_from" ,"deleted" ,"updated_at" ,"forked_commit_id"]
# projects = pd.read_csv(os.path.join(data_path,'projects.csv'), header=None, names=header_list, quotechar='"', escapechar='\\')
# projects['description'] = projects.parallel_apply(lambda row: clean(row['description'], no_line_breaks=True),axis = 1)
# projects.to_csv(os.path.join(filtered_dir,'easy_read/projects.csv'))

In [10]:
# chunksize = 10 ** 6
# header_list = ["id" ,"url" ,"owner_id" ,"name" ,"description" ,"language" ,"created_at" ,"forked_from" ,"deleted" ,"updated_at" ,"forked_commit_id"]

# header = True
# for chunk in tqdm(pd.read_csv(os.path.join(data_path,'projects.csv'), header=None, names=header_list, quotechar='"', escapechar='\\', chunksize=chunksize)):
#     chunk = chunk.drop('description', axis=1)
#     chunk.to_csv(os.path.join(filtered_dir,'easy_read/no_descp_projects.csv'),header=header, mode='a', index=False)
#     header = False

In [11]:
# pandarallel.initialize()
# chunksize = 10 ** 6
# header_list = ["pull_request_id","user_id","comment_id","position","body","commit_id","created_at"]

# header = True
# for chunk in tqdm(pd.read_csv(os.path.join(data_path,'pull_request_comments.csv'), header=None, names=header_list, quotechar='"', escapechar='\\', chunksize=chunksize)):
#     chunk['body'] = chunk.parallel_apply(lambda row: clean(row['body'], no_line_breaks=True),axis = 1)
#     chunk.to_csv(os.path.join(filtered_dir,'easy_read/pull_request_comments.csv'),header=header, mode='a', index=False)
#     header = False

# Load data

In [12]:
# header_list = "id int,sha string,author_id int,committer_id int,project_id int,created_at timestamp"
header_list = ["id","sha","author_id","committer_id","project_id","created_at"]
commits = ks.read_csv(os.path.join(data_path,'commits.csv'), header=None, names=header_list)

In [7]:
# header_list = "id int,url string,owner_id int,name string,description string,language string,created_at timestamp,forked_from int,deleted int,updated_at timestamp,forked_commit_id int"
# header_list = ["id" ,"url" ,"owner_id" ,"name" ,"language" ,"created_at" ,"forked_from" ,"deleted" ,"updated_at" ,"forked_commit_id"]
projects = ks.read_csv(os.path.join(filtered_dir,'easy_read/no_descp_projects.csv'))
# projects = spark_session.read.csv(os.path.join(data_path,'projects.csv'), multiLine=True,header=False)
# projects = ks.DataFrame(projects)
# projects.columns=header_list
# projects.write.csv(os.path.join(filtered_dir,'projects_easyread'))

In [12]:
sql = 'select * from {projects} where id = 213'
test = ks.sql(sql)
test.to_pandas()['url']

In [9]:
header_list = ["id","login","company","created_at","type","fake","deleted","long","lat","country_code","state","city","location"]
users = ks.read_csv(os.path.join(data_path,'users.csv'), header=None, names=header_list)

In [10]:
sql = 'select * from {users} where id = 1396688'
test = ks.sql(sql)
test.to_spark().show()

+-------+-----+-------+-------------------+----+----+-------+-----------+-----------+------------+-------------+------+---------------+
|     id|login|company|         created_at|type|fake|deleted|       long|        lat|country_code|        state|  city|       location|
+-------+-----+-------+-------------------+----+----+-------+-----------+-----------+------------+-------------+------+---------------+
|1396688|bl4de|     \N|2013-01-04 17:16:30| USR|   0|      0|-6.26749000|53.34410000|          IE|County Dublin|Dublin|Dublin, Ireland|
+-------+-----+-------+-------------------+----+----+-------+-----------+-----------+------------+-------------+------+---------------+



In [14]:
# # header_list ="project_id int, language string, bytes int, created_at timestamp"
# header_list =["project_id"," language"," bytes"," created_at timestamp"]
# project_languages = ks.read_csv(os.path.join(data_path,'project_languages.csv'), header=None, names=header_list)

# Filter out project in other language and deleted project and original project (not fork)

In [15]:
selected_language = ['C', 'C++', 'Java', 'JavaScript','Python', 'PHP', 'Ruby']

In [16]:
sql = 'select * from {projects} p where p.language in {selected_language} and p.deleted == 0 and forked_from=="N"'

In [17]:
filtered_projects = ks.sql(sql)

In [18]:
filtered_projects.to_csv(os.path.join(filtered_dir,'projects'))

In [8]:
selected_language = ['Python']

In [9]:
sql = 'select url, name from {projects} p where p.language in {selected_language} and p.deleted == 0 and forked_from=="N"'

In [10]:
filtered_projects = ks.sql(sql)

In [11]:
filtered_projects.to_csv(os.path.join(filtered_dir,'python_projects_name_url'),num_files=1)

# Reload projects

In [19]:
projects = ks.read_csv(os.path.join(filtered_dir,'projects/*.csv'))

# Calculate number of project's commits

In [20]:
sql = "select p.id as project_id, p.url, p.language, p.created_at as project_created_at , p.updated_at, c.id as commit_id, c.created_at as commit_created_at from {projects} p inner join {commits} c on c.project_id == p.id"

In [21]:
project_commits = ks.sql(sql)

In [22]:
project_commits.to_csv(os.path.join(filtered_dir,'project_commits'))

In [23]:
project_commits.to_spark().show()

+----------+--------------------+--------+-------------------+-------------------+---------+-------------------+
|project_id|                 url|language| project_created_at|         updated_at|commit_id|  commit_created_at|
+----------+--------------------+--------+-------------------+-------------------+---------+-------------------+
|       833|https://api.githu...|    Ruby|2010-09-16 15:34:02|2020-10-11 12:43:29|    15994|2012-06-09 06:28:50|
|       833|https://api.githu...|    Ruby|2010-09-16 15:34:02|2020-10-11 12:43:29|    15999|2012-06-09 05:12:00|
|       833|https://api.githu...|    Ruby|2010-09-16 15:34:02|2020-10-11 12:43:29|  1399920|2012-08-02 23:50:38|
|       833|https://api.githu...|    Ruby|2010-09-16 15:34:02|2020-10-11 12:43:29|  1400182|2012-08-02 23:51:37|
|       833|https://api.githu...|    Ruby|2010-09-16 15:34:02|2020-10-11 12:43:29|  1400186|2012-08-02 23:51:07|
|       833|https://api.githu...|    Ruby|2010-09-16 15:34:02|2020-10-11 12:43:29|  1400390|2012

In [24]:
project_commits = ks.read_csv(os.path.join(filtered_dir,'project_commits/*.csv'))

# Filter only project that have commit more than 500

In [25]:
sql = "select project_id, count(commit_id) from {project_commits} pc group by project_id having count(commit_id) > 500"

In [26]:
count_commit_project = ks.sql(sql)

In [27]:
count_commit_project.to_spark().show()

+----------+----------------+
|project_id|count(commit_id)|
+----------+----------------+
|       833|            1601|
|      1088|          117966|
|      3794|            2799|
|      3918|            3540|
|      4900|            1177|
|      6620|             525|
|      7880|             537|
|      7982|             914|
|      9427|           23743|
|      9900|             934|
|     10362|            4006|
|     13285|             554|
|     13289|            8307|
|     16861|            2149|
|     18800|            1518|
|     18944|            3168|
|     28577|            3060|
|     29894|            6176|
|     43688|            1238|
|     48254|            2585|
+----------+----------------+
only showing top 20 rows



In [28]:
selected_project = count_commit_project['project_id'].to_numpy()

In [29]:
selected_project = [int(i) for i in selected_project]

In [30]:
sql = "select * from {project_commits} cp where cp.project_id in {selected_project}"

In [31]:
project_commits = ks.sql(sql)

In [ ]:
project_commits.to_csv(os.path.join(filtered_dir,'project_commits_500commit'))

In [ ]:
project_commits = ks.read_csv(os.path.join(filtered_dir,'project_commits_500commit/*.csv'))

# most active 2 year filter

In [ ]:
sql = "select project_id, YEAR(commit_created_at), count(commit_id) from {project_commits} pc group by project_id, YEAR(commit_created_at) order by project_id, YEAR(commit_created_at)"

In [ ]:
count_commit_project = ks.sql(sql)

In [ ]:
count_commit_project.to_spark().show()

In [ ]:
count_commit_project_year_df = count_commit_project.to_pandas()

In [ ]:
current_proj = 3
current_count = 0
remove_project = []
for i, row in tqdm(count_commit_project_year_df.iterrows(),total= count_commit_project_year_df.shape[0]):
    if row['project_id'] != current_proj:
        if current_count < 2:
            remove_project.append(current_proj)
        current_proj = row['project_id']
        current_count = 0
    current_count = current_count + (1 if row['count(commit_id)'] > 100 else 0)

In [ ]:
remove_project = [int(i) for i in remove_project]

In [ ]:
sql = "select * from {project_commits} cp where cp.project_id not in {remove_project}"

In [ ]:
project_commits = ks.sql(sql)

In [ ]:
project_commits.to_csv(os.path.join(filtered_dir,'project_commits_500commit_100mostactive2'))

In [ ]:
project_commits = ks.read_csv(os.path.join(filtered_dir,'project_commits_500commit_100mostactive2/*.csv'))

# Candidate project

In [66]:
sql = "select distinct project_id from {project_commits}"

In [67]:
candidate_project = ks.sql(sql)

In [ ]:
candidate_project = [int(i) for i in candidate_project.to_pandas()['project_id']]

In [20]:
len(candidate_project)

72635

In [17]:
with open(os.path.join(filtered_dir,"candidate_projects"),'wb') as outfile:
    pickle.dump(candidate_project,outfile)


In [7]:
with open(os.path.join(filtered_dir,"candidate_projects"),'rb') as infile:
    candidate_project = pickle.load(infile)


# Filter pull_request (make file smaller only candidate project pull reqeust)

In [ ]:
header_list = ["id","head_repo_id","base_repo_id","head_commit_id","base_commit_id","pullreq_id","intra_branch"]
pull_requests = ks.read_csv(os.path.join(data_path,'pull_requests.csv'), header=None, names=header_list)

In [ ]:
header_list = ["id","pull_request_id","created_at","action","actor_id"]
pull_request_history = ks.read_csv(os.path.join(data_path,'pull_request_history.csv'), header=None, names=header_list)

In [ ]:
# header_list = ["pull_request_id","user_id","comment_id","position","body","commit_id","created_at"]
pull_request_comments = ks.read_csv(os.path.join(filtered_dir,'easy_read/pull_request_comments.csv'))
# pull_request_comments = spark_session.read.option("header",False).option("multiline",True).csv(os.path.join(data_path,'pull_request_comments.csv'))
# pull_request_comments = ks.DataFrame(pull_request_comments)
# pull_request_comments.columns=header_list
# pull_request_comments.to_csv(os.path.join(filtered_dir,'pull_request_comments_easy_read/pull_request_comments.csv'))

In [ ]:
sql = 'select * from {pull_requests} pr where pr.base_repo_id in {candidate_project}'

In [ ]:
pull_requests = ks.sql(sql)

In [ ]:
pull_requests.to_csv(os.path.join(filtered_dir,'pull_requests'))

In [ ]:
pull_requests = ks.read_csv(os.path.join(filtered_dir,'pull_requests/*.csv'))

# Filter issue (make file smaller only candidate project issue)

In [8]:
header_list = ["id","repo_id","reporter_id","assignee_id","pull_request","pull_request_id","created_at","issue_id"]
issues = ks.read_csv(os.path.join(data_path,'issues.csv'), header=None, names=header_list)

In [9]:
sql = 'select * from {issues} where repo_id in {candidate_project}'

In [10]:
issues = ks.sql(sql)

In [11]:
issues.to_csv(os.path.join(filtered_dir,'issues'))

In [ ]:
issues = ks.read_csv(os.path.join(filtered_dir,'issues/*.csv'))

# ETC

In [ ]:
sql = "select language,distinct project_id project_id from {project_commits}"

In [ ]:
project_language = ks.sql(sql)

In [ ]:
project_language.to_csv(os.path.join(filtered_dir,'project_language'))

In [73]:
df

,language,project_id
0,JavaScript,18504
1,C++,9052
2,C,8071
3,PHP,7008
4,Ruby,4363
5,Python,14102
6,Java,11535
